In [49]:
pip install open3d numpy

In [50]:
!pip install mealpy

In [51]:
import urllib.request
import zipfile
import os
from tqdm import tqdm

# URL and target paths
url = "https://modelnet.cs.princeton.edu/ModelNet40.zip"
zip_path = "ModelNet40.zip"
extract_path = "ModelNet40"

# Download with progress bar
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

print("📥 Downloading ModelNet40.zip...")
with DownloadProgressBar(unit='B', unit_scale=True, miniters=1, desc=zip_path) as t:
    urllib.request.urlretrieve(url, filename=zip_path, reporthook=t.update_to)

# Extract with progress bar
print("📦 Extracting ModelNet40.zip...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    for file in tqdm(file_list, desc="Extracting", unit="files"):
        zip_ref.extract(file, extract_path)

print("✅ Done! Files extracted to:", extract_path)


📥 Downloading ModelNet40.zip...


ModelNet40.zip: 0.00B [00:00, ?B/s]

ModelNet40.zip:   8%|▊         | 166M/2.04G [00:37<07:07, 4.38MB/s]    


KeyboardInterrupt: 

In [59]:
import open3d as o3d

mesh = o3d.io.read_triangle_mesh("sodaCan.glb")
mesh.compute_vertex_normals()
center = mesh.get_center()
mesh.translate(-center)
bbox = mesh.get_axis_aligned_bounding_box()
scale = 1.0 / max(bbox.get_extent())
mesh.scale(scale, center=(0, 0, 0))
print("Original mesh center:", center)

Original mesh center: [  0.56459773 121.17310794  -2.60005345]


In [53]:
import numpy as np

# --- Spherical candidates ---
def fibonacci_sphere(samples=120, radius=1):
    points = []
    phi = np.pi * (3. - np.sqrt(5.))
    for i in range(samples):
        y = 1 - (i / float(samples - 1)) * 2
        r = np.sqrt(1 - y * y)
        theta = phi * i
        x = np.cos(theta) * r
        z = np.sin(theta) * r
        points.append([x * radius, y * radius, z * radius])
    return np.array(points)

sphere_views = fibonacci_sphere(samples=120, radius=1)

# --- Surface-normal-driven candidates ---
def generate_normal_based_views(mesh, num_views=120, distance=0.3):
    pcd = mesh.sample_points_uniformly(number_of_points=num_views)
    pcd.estimate_normals()
    points = np.asarray(pcd.points)
    normals = np.asarray(pcd.normals)
    candidate_views = points - normals * distance
    return candidate_views

normal_views = generate_normal_based_views(mesh, num_views=120, distance=0.3)

# --- Combine all candidates ---
candidate_views = np.vstack([sphere_views, normal_views])
print(f"Total candidate views: {len(candidate_views)}")


Total candidate views: 240


In [54]:
from tqdm import tqdm
import open3d as o3d

pcd = mesh.sample_points_uniformly(number_of_points=5000)
# --- Helper: Normal-aware coverage per view ---
def points_covered_by_view(view_pos, pcd, normals, radius=0.6, angle_thresh_deg=60):
    _, pt_map = pcd.hidden_point_removal(view_pos, radius)
    pts = np.asarray(pcd.points)[pt_map]
    nrm = np.asarray(normals)[pt_map]
    view_dirs = pts - view_pos
    view_dirs = view_dirs / np.linalg.norm(view_dirs, axis=1, keepdims=True)
    dots = (nrm * view_dirs).sum(axis=1)
    angles = np.arccos(np.clip(dots, -1, 1)) * 180 / np.pi
    covered_indices = np.array(pt_map)[angles < angle_thresh_deg]
    return set(covered_indices)

# --- Generate normal-aware coverage sets for all candidate views ---
view_coverage = []
for view in candidate_views:
    covered = points_covered_by_view(
        view, pcd, pcd.normals, radius=0.6, angle_thresh_deg=60
    )
    view_coverage.append(covered)
total_points = set(range(len(pcd.points)))


In [55]:
import numpy as np
from tqdm import tqdm
import open3d as o3d
from mealpy import FloatVar, SMA

# --- Step 1: Prepare mesh, pcd, and candidate views (assume already done) ---
# mesh = ... (your normalized mesh)
# candidate_views = ... (e.g., sphere + normal-based, shape = [N, 3])
# pcd = mesh.sample_points_uniformly(number_of_points=5000)
# pcd.estimate_normals()

# --- Step 2: Normal-aware coverage helper ---
def points_covered_by_view(view_pos, pcd, normals, radius=0.6, angle_thresh_deg=60):
    _, pt_map = pcd.hidden_point_removal(view_pos, radius)
    pts = np.asarray(pcd.points)[pt_map]
    nrm = np.asarray(normals)[pt_map]
    view_dirs = pts - view_pos
    view_dirs = view_dirs / np.linalg.norm(view_dirs, axis=1, keepdims=True)
    dots = (nrm * view_dirs).sum(axis=1)
    angles = np.arccos(np.clip(dots, -1, 1)) * 180 / np.pi
    covered_indices = np.array(pt_map)[angles < angle_thresh_deg]
    return set(covered_indices)

# --- Step 3: Compute normal-aware coverage for candidate views ---
view_coverage = []
for view in tqdm(candidate_views, desc="Normal-aware coverage"):
    covered = points_covered_by_view(
        view, pcd, pcd.normals, radius=0.6, angle_thresh_deg=60
    )
    view_coverage.append(covered)
total_points = set(range(len(pcd.points)))

# --- Step 4: SMA + Greedy initial selection ---
init_k = 10
coverage_goal = 0.97
view_budget = 60

def sma_objective(solution):
    indices = [int(round(x)) % len(view_coverage) for x in solution]
    indices = list(set(indices))
    covered = set()
    for i in indices:
        covered |= view_coverage[i]
    uncovered = len(total_points - covered)
    return uncovered

problem_dict = {
    "obj_func": sma_objective,
    "minmax": "min",
    "bounds": FloatVar(
        lb=(0,) * init_k,
        ub=(len(view_coverage) - 1,) * init_k,
        name="view_index"
    )
}

model = SMA.DevSMA(epoch=200, pop_size=20, p_t=0.03)
g_best = model.solve(problem_dict)

sma_selected = set(int(round(x)) % len(view_coverage) for x in g_best.solution)
covered = set().union(*[view_coverage[i] for i in sma_selected])
print(f"SMA: {len(sma_selected)} views, initial normal-aware coverage = {len(covered)/len(total_points)*100:.2f}%")

# --- Step 5: Greedy fill-in for remaining coverage ---
selected = set(sma_selected)
while len(selected) < view_budget and len(covered)/len(total_points) < coverage_goal:
    best_gain = 0
    best_idx = None
    for i in range(len(view_coverage)):
        if i in selected:
            continue
        gain = len(view_coverage[i] - covered)
        if gain > best_gain:
            best_gain = gain
            best_idx = i
    if best_idx is not None and best_gain > 0:
        selected.add(best_idx)
        covered |= view_coverage[best_idx]
    else:
        break

coverage_percent = len(covered) / len(total_points) * 100
print(f"\nHybrid SMA+Greedy: {len(selected)} views, normal-aware coverage = {coverage_percent:.2f}%")
print(f"Selected indices: {sorted(selected)}")

# --- Step 6: Adaptive NBV generation for missed points ---
missed_indices = list(total_points - covered)
missed_points = np.asarray(pcd.points)[missed_indices]
missed_normals = np.asarray(pcd.normals)[missed_indices]

adaptive_distance = 0.3
adaptive_nbvs = missed_points - missed_normals * adaptive_distance

# Compute coverage for adaptive NBVs
adaptive_coverage = []
for view in tqdm(adaptive_nbvs, desc="Adaptive NBVs coverage"):
    covered_adaptive = points_covered_by_view(
        view, pcd, pcd.normals, radius=0.6, angle_thresh_deg=60
    )
    adaptive_coverage.append(covered_adaptive)

# Greedily add adaptive NBVs until coverage_goal or budget is hit
adaptive_selected = set()
current_covered = set(covered)
max_adaptive_nbvs = 30  # adjust as desired

while len(current_covered) / len(pcd.points) < coverage_goal and len(adaptive_selected) < max_adaptive_nbvs:
    best_gain = 0
    best_idx = None
    for i in range(len(adaptive_coverage)):
        if i in adaptive_selected:
            continue
        gain = len(adaptive_coverage[i] - current_covered)
        if gain > best_gain:
            best_gain = gain
            best_idx = i
    if best_idx is not None and best_gain > 0:
        adaptive_selected.add(best_idx)
        current_covered |= adaptive_coverage[best_idx]
    else:
        break

adaptive_coverage_percent = len(current_covered) / len(pcd.points) * 100
print(f"\nAdaptive NBV added: {len(adaptive_selected)} new views, total normal-aware coverage = {adaptive_coverage_percent:.2f}%")

# Combine all NBV indices for optional visualization
all_nbv_positions = np.vstack([candidate_views, adaptive_nbvs])
all_selected_indices = list(selected) + [len(candidate_views) + i for i in adaptive_selected]

# --- Optional: visualize selected NBVs ---
spheres = []
for i, pos in enumerate(all_nbv_positions):
    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.02)
    sphere.translate(pos)
    if i in all_selected_indices:
        sphere.paint_uniform_color([1, 0, 0])  # red for selected
    else:
        sphere.paint_uniform_color([0.7, 0.7, 0.7])
    spheres.append(sphere)
o3d.visualization.draw_geometries([mesh] + spheres)


Normal-aware coverage: 100%|██████████| 240/240 [00:01<00:00, 122.55it/s]
2025/07/15 11:46:55 PM, INFO, mealpy.bio_based.SMA.DevSMA: DevSMA(epoch=200, pop_size=20, p_t=0.03)
2025/07/15 11:46:55 PM, INFO, mealpy.bio_based.SMA.DevSMA: >>>Problem: P, Epoch: 1, Current best: 809.0, Global best: 809.0, Runtime: 0.02402 seconds
2025/07/15 11:46:55 PM, INFO, mealpy.bio_based.SMA.DevSMA: >>>Problem: P, Epoch: 2, Current best: 809.0, Global best: 809.0, Runtime: 0.02292 seconds
2025/07/15 11:46:55 PM, INFO, mealpy.bio_based.SMA.DevSMA: >>>Problem: P, Epoch: 3, Current best: 792.0, Global best: 792.0, Runtime: 0.02754 seconds
2025/07/15 11:46:55 PM, INFO, mealpy.bio_based.SMA.DevSMA: >>>Problem: P, Epoch: 4, Current best: 792.0, Global best: 792.0, Runtime: 0.02311 seconds
2025/07/15 11:46:55 PM, INFO, mealpy.bio_based.SMA.DevSMA: >>>Problem: P, Epoch: 5, Current best: 792.0, Global best: 792.0, Runtime: 0.01790 seconds
2025/07/15 11:46:55 PM, INFO, mealpy.bio_based.SMA.DevSMA: >>>Problem: P, Ep

SMA: 9 views, initial normal-aware coverage = 85.98%

Hybrid SMA+Greedy: 55 views, normal-aware coverage = 89.50%
Selected indices: [0, 3, 5, 6, 9, 15, 18, 20, 27, 30, 32, 33, 37, 38, 39, 41, 47, 49, 50, 51, 61, 68, 71, 72, 79, 82, 84, 85, 86, 88, 89, 91, 92, 94, 95, 97, 98, 113, 121, 129, 145, 146, 152, 153, 156, 161, 171, 183, 205, 206, 216, 223, 230, 232, 239]


Adaptive NBVs coverage: 100%|██████████| 525/525 [00:03<00:00, 155.77it/s]



Adaptive NBV added: 29 new views, total normal-aware coverage = 94.12%


In [56]:
import open3d as o3d
import numpy as np

# --- After NBV selection ---
# `current_covered` contains indices of covered points after adaptive NBV
all_indices = set(range(len(pcd.points)))
missed_indices = list(all_indices - current_covered)
covered_indices = list(current_covered)

# Covered points & normals (green)
covered_points = np.asarray(pcd.points)[covered_indices]
covered_normals = np.asarray(pcd.normals)[covered_indices]
covered_pcd = o3d.geometry.PointCloud()
covered_pcd.points = o3d.utility.Vector3dVector(covered_points)
covered_pcd.normals = o3d.utility.Vector3dVector(covered_normals)
covered_pcd.paint_uniform_color([0, 1, 0])  # green

# Missed points & normals (red)
missed_points = np.asarray(pcd.points)[missed_indices]
missed_normals = np.asarray(pcd.normals)[missed_indices]
missed_pcd = o3d.geometry.PointCloud()
missed_pcd.points = o3d.utility.Vector3dVector(missed_points)
missed_pcd.normals = o3d.utility.Vector3dVector(missed_normals)
missed_pcd.paint_uniform_color([1, 0, 0])  # red

# Visualize mesh, covered (green) and missed (red) points with normals as arrows
o3d.visualization.draw_geometries(
    [mesh, covered_pcd, missed_pcd],
    point_show_normal=True
)
n_covered = len(covered_indices)
n_missed = len(missed_indices)
n_total = len(pcd.points)

print(f"Covered points/normals: {n_covered} / {n_total} ({n_covered/n_total*100:.2f}%)")
print(f"Missed points/normals:  {n_missed} / {n_total} ({n_missed/n_total*100:.2f}%)")


Covered points/normals: 4706 / 5000 (94.12%)
Missed points/normals:  294 / 5000 (5.88%)


In [60]:
import numpy as np
import json

# --- Example (replace these with your actual variables) ---
# all_nbv_positions: shape (N, 3), numpy array of all candidate NBVs
# all_selected_indices: list of indices (in all_nbv_positions) for selected NBVs

selected_nbv_positions = all_nbv_positions[all_selected_indices]

# Convert to a list of dicts for clarity
nbv_list = [
    {"id": int(i), "x": float(pos[0]), "y": float(pos[1]), "z": float(pos[2])}
    for i, pos in zip(all_selected_indices, selected_nbv_positions)
]

# Save to a nicely formatted JSON file
with open("nbv_points.json", "w") as f:
    json.dump({"nbv_points": nbv_list}, f, indent=2)

print("✅ Saved selected NBV points to nbv_points.json")


✅ Saved selected NBV points to nbv_points.json
